## Import

In [2]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import timm
from torchvision.models import video
import time

import warnings
warnings.filterwarnings(action='ignore') 

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [4]:
class CFG:
    model_name= "s3d"
    n_folds = 5
    n_classes = 13
    video_length=50
    img_size=256
    epochs=50
    lr=3e-4
    batch_size=16
    seed=41
        

## Fixed RandomSeed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG.seed) # Seed 고정

## Data Load

In [6]:
df = pd.read_csv('./train.csv')

## Train / Validation Split

In [7]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG.seed)

## CustomDataset

In [8]:
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_list):
        self.video_path_list = video_path_list
        self.label_list = label_list
        
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        cap = cv2.VideoCapture(path)
        for _ in range(CFG.video_length):
            _, img = cap.read()
            img = cv2.resize(img, (CFG.img_size, CFG.img_size))
            img = img / 255.
            frames.append(img)
        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)

In [9]:
train_dataset = CustomDataset(train['video_path'].values, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG.batch_size, shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['video_path'].values, val['label'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=0)

## Model Define

In [30]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=CFG.n_classes, fc_type='shallow'):
        super(BaseModel, self).__init__()
        self.fc_type=fc_type
        self.num_classes=num_classes
        
        #get backbone
        self.backbone=getattr(video,CFG.model_name)(pretrained=True)
        #self.fc=self.get_fc()
        
        self.backbone.classifier[1]=nn.Conv3d(1024, 13, kernel_size=(1, 1, 1), stride=(1, 1, 1))
        
    def get_fc(self):
        
        if self.fc_type == 'shallow':
            fc = nn.Linear(400, self.num_classes)
            
        else:
            raise ValueError(f"Wrong fc-type input {self.fc_type}")
        return fc
    
    def forward(self, x):
        x = self.backbone(x)
        #x= self.fc(x)
        
        return x

In [29]:
from torchsummary import summary
model=BaseModel().to(device)
summary(model, (3,50,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1     [-1, 64, 50, 128, 128]           9,408
       BatchNorm3d-2     [-1, 64, 50, 128, 128]             128
              ReLU-3     [-1, 64, 50, 128, 128]               0
            Conv3d-4     [-1, 64, 25, 128, 128]          28,672
       BatchNorm3d-5     [-1, 64, 25, 128, 128]             128
              ReLU-6     [-1, 64, 25, 128, 128]               0
         MaxPool3d-7       [-1, 64, 25, 64, 64]               0
            Conv3d-8       [-1, 64, 25, 64, 64]           4,096
       BatchNorm3d-9       [-1, 64, 25, 64, 64]             128
             ReLU-10       [-1, 64, 25, 64, 64]               0
           Conv3d-11      [-1, 192, 25, 64, 64]         110,592
      BatchNorm3d-12      [-1, 192, 25, 64, 64]             384
             ReLU-13      [-1, 192, 25, 64, 64]               0
           Conv3d-14      [-1, 192, 25,

## Train

In [31]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_score = 0
    best_model = None
    cnt=0
    for epoch in range(1, CFG.epochs+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(videos)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
            torch.save(model.state_dict(), './weights/s3d.pt')
            cnt=0
        else:
            print("early stopping count : {}".format(cnt))
            cnt+=1
        
        if cnt==5:
            print("early stopping done")
            break
            
    return best_model

In [32]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            logit = model(videos)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

## Run!!

In [33]:
model = BaseModel(fc_type='shallow')
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG.lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.00001)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Let's use 2 GPUs!


  0%|          | 0/135 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.71333] Val Loss : [2.45726] Val F1 : [0.06607]


  0%|          | 0/135 [00:00<?, ?it/s]

...

  0%|          | 0/34 [00:00<?, ?it/s]

Epoch [13], Train Loss : [0.06283] Val Loss : [1.22572] Val F1 : [0.23137]
early stopping count : 3


  0%|          | 0/135 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

Epoch [14], Train Loss : [0.02511] Val Loss : [0.85209] Val F1 : [0.29892]
early stopping count : 4
early stopping done


## Inference

In [34]:
test = pd.read_csv('./test.csv')

In [35]:
test_dataset = CustomDataset(test['video_path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG.batch_size, shuffle=False, num_workers=0)

In [36]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [37]:
preds = inference(model, test_loader, device)

  0%|          | 0/113 [00:00<?, ?it/s]

## Submission

In [38]:
submit = pd.read_csv('./sample_submission.csv')

In [39]:
submit['label'] = preds
submit.head()

,sample_id,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0


In [40]:
submit.to_csv('./s3d_submit.csv', index=False)